In [ ]:
import utils_data as utils

import torch
import torch.nn as nn 
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torch.utils.data import WeightedRandomSampler
from torch.optim import Adam

from tqdm import tqdm
from sklearn.metrics import accuracy_score
from collections import Counter
import numpy as np
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"

### Importation du dataset

In [ ]:
# Solution pour randomiser la séparation train / val
text_all = utils.get_data()
labels_all = utils.get_labels()
limit = int(len(labels_all) * 0.8) + 1

merged_all = utils.merge_data_labels(text_all, labels_all)
np.random.shuffle(merged_all)

merged_train, merged_val = merged_all[:limit], merged_all[limit:] # Séparation entre les données de train et de validation (80% - 20%)

text_test = utils.get_test()

### Hyperparamètres

In [ ]:
nb_RNN = 3
limit_RNN = int(len(merged_train) * 0.8) + 1

target_classes = ["negative", "neutral", "positive"]
max_words = 75
embed_len = 75
hidden_dim = 75
n_layers = 5
p_dropout = 0.3
epochs = 15
learning_rate = 0.001

### Séparation des données pour chaque RNN

In [ ]:
# Chaque RNN sera entraîné sur un ensemble de données différent
merged_train_set = []
for i in range(nb_RNN):
  np.random.shuffle(merged_train)
  merged_train_set.append(merged_train[:limit_RNN])

Note importante : toute la suite est inspiré de : https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-rnn-for-text-classification-tasks, ainsi que du cours IFT 6135-A2022.

### Création de tokens à partir des phrases

In [ ]:
# On construit notre vocabulaire à partir des mots de la base de données
tokenizer = get_tokenizer("basic_english")

def build_vocabulary(datasets):
    for dataset in datasets:
        for text in dataset:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocabulary([text_all]), min_freq=1, specials=["<UNK>"])
vocab.set_default_index(vocab["<UNK>"])

### Préparation des données

In [ ]:
# Comptage du nombre d'exemples par classe pour établir des poids et un "sampler" (sur l'ensemble des données d'entraînement choisies)
samplers = []
for j in range(nb_RNN):
    label_train = np.asarray([list(e) for e in merged_train_set[j]])[:, 1].astype(int)
    counter = Counter(label_train.tolist())

    class_weights = [len(label_train) / counter[i] for i in range(len(counter))]
    weights = [class_weights[label_train[i]] for i in range(len(label_train))]
    samplers.append(WeightedRandomSampler(torch.DoubleTensor(weights), len(label_train)))

In [ ]:
# Préparation des données sous un format lisible par PyTorch
data_train_experts = []
data_val = to_map_style_dataset(merged_val)
data_test = to_map_style_dataset(text_test)
for i in range(nb_RNN):
  data_train_experts.append(to_map_style_dataset(merged_train_set[i]))

# Fonction servant à transformer nos données (textes) sous formes de nombres.
def vectorize_batch(batch):
    X, Y = list(zip(*batch))
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y, dtype = torch.long)

# Fonction servant à transformer nos données (textes) sous formes de nombres (dans le cas de données de test, on a pas de labels)
def vectorize_test_batch(batch):
    X = list(batch)
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32)

# Préparation des DataLoader pouvant être lu dans les modèles PyTorch
train_loader_experts = []
val_loader = DataLoader(data_val, batch_size=1024, collate_fn=vectorize_batch)
test_loader = DataLoader(data_test, batch_size=1024, collate_fn=vectorize_test_batch)

for i in range(nb_RNN):
  train_loader_experts.append(DataLoader(data_train_experts[i], batch_size=1024, collate_fn=vectorize_batch, sampler=samplers[i]))

### Définition du RNN (GRU)

In [ ]:
# Architecture de notre GRU
class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.dropout = nn.Dropout(p_dropout)
        self.rnn = nn.GRU(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(target_classes))

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch).to(device)
        output, hidden = self.rnn(self.dropout(embeddings))
        return F.softmax(self.linear(output[:,-1]), dim=-1)

### Entraînement des RNN

In [ ]:
# Fonction d'évaluation des RNN
def evaluate(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_losses_all, Y_preds_all = [], []
        for i in range(nb_RNN):
          model[i].eval()
          Y_true, Y_preds, losses = [],[],[]
          for X, Y in val_loader:
              Y_true.append(Y)
              Y = torch.tensor(utils.to_one_hot(Y.numpy(), len(target_classes)))
              X, Y = X.to(device), Y.to(device)
              preds = model[i](X)
              loss = loss_fn(preds, Y)
              losses.append(loss.item())
              Y_preds.append(preds)

          Y_true = torch.cat(Y_true)
          Y_preds = torch.cat(Y_preds)

          Y_losses_all.append(torch.tensor(losses).mean())
          Y_preds_all.append(Y_preds)

          model[i].train()

        Y_losses_all = torch.tensor(Y_losses_all).mean()
        Y_preds_all = torch.mode(torch.stack(Y_preds_all, dim = -1).argmax(dim = -2))[0]

        print("Valid Loss : {:.3f} | Valid Acc : {:.3f}".format(Y_losses_all, accuracy_score(Y_true.detach().cpu().numpy(), Y_preds_all.detach().cpu().numpy())))

# Fonction d'entraînement du RNN        
def train(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(1, epochs+1):
      Y_losses_all, Y_acc_all = [], []
      for i in range(nb_RNN):
        Y_true, Y_preds, losses = [],[],[]
        for X, Y in tqdm(train_loader[i]):
            Y_true.append(Y)
            Y = torch.tensor(utils.to_one_hot(Y.numpy(), len(target_classes)))
            X, Y = X.to(device), Y.to(device)
            optimizer[i].zero_grad()
            preds = model[i](X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())
            Y_preds.append(preds.argmax(dim = -1))
            loss.backward()
            optimizer[i].step()

        Y_true = torch.cat(Y_true)
        Y_preds = torch.cat(Y_preds)

        Y_losses_all.append(torch.tensor(losses).mean())
        Y_acc_all.append(accuracy_score(Y_true.detach().cpu().numpy(), Y_preds.detach().cpu().numpy()))

      Y_losses_all = torch.tensor(Y_losses_all).mean()
      Y_acc_all = torch.tensor(Y_acc_all).mean()
            
      print("Epoch {} | Train Loss (moy) : {:.3f} | Train acc (moy) : {:.3f}".format(epoch, Y_losses_all, Y_acc_all))
      evaluate(model, loss_fn, val_loader)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
rnn_classifiers, optimizers = [], []
for i in range(nb_RNN):
  rnn_classifiers.append(RNNClassifier().to(device))
  optimizers.append(Adam(rnn_classifiers[i].parameters(), lr=learning_rate))

train(rnn_classifiers, loss_fn, optimizers, train_loader_experts, val_loader, epochs)

### Prédictions sur l'ensemble de test

In [ ]:
def predict(model, loader):
    Y_preds_experts = []
    for i in range(nb_RNN):
      Y_preds = []
      for X in tqdm(loader):
          X = X.to(device)
          preds = model[i](X)
          Y_preds.append(preds.detach().cpu())
      gc.collect()
      Y_preds = torch.cat(Y_preds)
    Y_preds_experts.append(Y_preds)
    return torch.mode(torch.stack(Y_preds_experts, dim = -1).argmax(dim = -2))[0].numpy()

Y_preds = predict(rnn_classifiers, test_loader)

### Enregistrement des résultats

In [ ]:
utils.save_results(Y_preds, "RNN_ensemble")